# Importing data
Data extracted from:
- https://www.kaggle.com/datasets/nikdavis/steam-store-games
- https://www.kaggle.com/datasets/tamber/steam-video-games

In [5]:
import pandas as pd

user_time_played = pd.read_csv('./data/steam-200k.csv', header = None)

In [6]:
user_time_played.drop([4], axis = 1, inplace=True)
user_time_played.columns = ['user_id', 'game_name', 'action', 'hours']
user_time_played.head()

,user_id,game_name,action,hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [26]:
game_description = pd.read_csv('./data/steam_description_data.csv')
game_media = pd.read_csv('./data/steam_media_data.csv')
game_requirements = pd.read_csv('./data/steam_requirements_data.csv')
game_support = pd.read_csv('./data/steam_support_info.csv')
game_steam = pd.read_csv('./data/steam.csv')

In [27]:
game_steam=game_steam.rename(columns = {'appid':'steam_appid'})
game_steam.head()

,steam_appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [28]:
import pandas as pd
from functools import reduce

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['steam_appid'],
                                            how='outer'), [game_steam, game_description, game_media, game_requirements, game_support])

In [30]:
df_consolidado = df_merged.head()

: 

## Requisition from steam api

In [3]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests


In [4]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))


def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data



In [12]:
page = 1
url = "https://steamspy.com/api.php"
parameters = {"request": "all", 'page' : page}

# request 'all' from steam spy and parse into dataframe
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

# export disabled to keep consistency across download sessions
app_list.to_csv('./data/download/app_list.csv', index=False)

# instead read from stored csv
app_list = pd.read_csv('./data/download/app_list.csv')

# display first few rows
app_list.head()

,appid,name
0,1200,Red Orchestra: Ostfront 41-45
1,1510,Uplink
2,1520,DEFCON
3,1530,Multiwinia
4,1900,Earth 2160


In [13]:

# Set file parameters
download_path = './data/download'
steam_app_data = 'steam_app_data'+str(page+1)+'.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    batchsize=5
)

Starting at index 0:

Exported lines 0-4 to steam_app_data2.csv. Batch 0 time: 0:00:08 (avg: 0:00:08, remaining: 0:26:58)
Exported lines 5-9 to steam_app_data2.csv. Batch 1 time: 0:00:08 (avg: 0:00:08, remaining: 0:26:23)
Exported lines 10-14 to steam_app_data2.csv. Batch 2 time: 0:00:08 (avg: 0:00:08, remaining: 0:26:40)
Exported lines 15-19 to steam_app_data2.csv. Batch 3 time: 0:00:08 (avg: 0:00:08, remaining: 0:26:34)
Exported lines 20-24 to steam_app_data2.csv. Batch 4 time: 0:00:08 (avg: 0:00:08, remaining: 0:26:23)
Exported lines 25-29 to steam_app_data2.csv. Batch 5 time: 0:00:08 (avg: 0:00:08, remaining: 0:26:12)
Exported lines 30-34 to steam_app_data2.csv. Batch 6 time: 0:00:08 (avg: 0:00:08, remaining: 0:25:59)
Exported lines 35-39 to steam_app_data2.csv. Batch 7 time: 0:00:08 (avg: 0:00:08, remaining: 0:25:48)
Exported lines 40-44 to steam_app_data2.csv. Batch 8 time: 0:00:09 (avg: 0:00:08, remaining: 0:25:52)
Exported lines 45-49 to steam_app_data2.csv. Batch 9 time: 0:00:

In [15]:
for page in range(2, 16):
    url = "https://steamspy.com/api.php"
    parameters = {"request": "all", 'page' : page}

    # request 'all' from steam spy and parse into dataframe
    json_data = get_request(url, parameters=parameters)
    steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

    # generate sorted app_list from steamspy data
    app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

    # export disabled to keep consistency across download sessions
    app_list.to_csv('./data/download/app_list'+str(page+1)+'.csv', index=False)

    # instead read from stored csv
    app_list = pd.read_csv('./data/download/app_list'+str(page+1)+'.csv')

    # Set file parameters
    download_path = './data/download'
    steam_app_data = 'steam_app_data'+str(page+1)+'.csv'
    steam_index = 'steam_index.txt'

    steam_columns = [
        'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
        'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
        'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
        'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
        'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
        'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
        'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
        'background', 'content_descriptors'
    ]

    # Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
    reset_index(download_path, steam_index)

    # Retrieve last index downloaded from file
    index = get_index(download_path, steam_index)

    # Wipe or create data file and write headers if index is 0
    prepare_data_file(download_path, steam_app_data, index, steam_columns)

    # Set end and chunksize for demonstration - remove to run through entire app list
    process_batches(
        parser=parse_steam_request,
        app_list=app_list,
        download_path=download_path,
        data_filename=steam_app_data,
        index_filename=steam_index,
        columns=steam_columns,
        begin=index
    )

Starting at index 0:

Exported lines 0-99 to steam_app_data3.csv. Batch 0 time: 0:02:09 (avg: 0:02:09, remaining: 0:21:32)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 100-199 to steam_app_data3.csv. Batch 1 time: 0:02:35 (avg: 0:02:22, remaining: 0:21:17)
Exported lines 200-299 to steam_app_data3.csv. Batch 2 time: 0:02:14 (avg: 0:02:19, remaining: 0:18:33)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 300-399 to steam_app_data3.csv. Batch 3 time: 0:02:43 (avg: 0:02:25, remaining: 0:16:56)
Exported lines 400-499 to steam_app_data3.csv. Batch 4 time: 0:02:12 (avg: 0:02:23, remaining: 0:14:15)
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
Exported lines 500-599 to steam_app_data3.csv. Batch 5 t

Bad pipe message: %s [b'', b'Q\x1c\xec\x02\xbdquEH\x86\n\x88?]\xb2$ \x06\xaa\x8ab#\xc0W\xfa\xec\x85\xbd.0\x83v9\xe2\xd2\x9e\x9d\xbb9\xdf;U\x00\x9d\x81Y\x089\x96\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00']
Bad pipe message: %s [b"\x99\xa3LNF\xf57\xcdL\x8fZ\xc0\xba\x13\xd2\x85\xd0\xd7\x00\x00\xf4\xc00\xc0,\xc0(\xc0$\xc0\x14\xc0\n\x00\xa5\x00\xa3\x00\xa1\x00\x9f\x00k\x00j\x00i\x00h\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00\xa7\x00m\x00:\x00\x89\xc02\xc0.\xc0*\xc0&\xc0\x0f\xc0\x05\x00\x9d\x00=\x005\x00\x84\xc0/\xc0+\xc0'\xc0#\xc0\x13\xc0\t\x00\xa4\x00\xa2\x00\xa0\x00\x9e\x00g\x0

No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.


In [14]:
range(2,10)

range(2, 10)